# Python Inventory System - Part 1: Data Models

This notebook covers the data models used in our Inventory Management System, focusing on the `models.py` file. We'll learn how to define database models using SQLModel, which is a modern Python library that combines SQLAlchemy and Pydantic.

## What We'll Cover

1. Introduction to Object-Relational Mapping (ORM)
2. SQLModel basics
3. Creating our Product model
4. Understanding model fields and options
5. Working with model instances

## 1. Introduction to Object-Relational Mapping (ORM)

**Object-Relational Mapping (ORM)** is a programming technique that lets you interact with your database using Python objects instead of writing raw SQL queries. This makes your code more Pythonic and type-safe.

**Benefits of using an ORM:**
- Write Python code instead of SQL
- Work with Python objects instead of database rows
- Type checking and validation
- Database-agnostic code (can switch between SQLite, PostgreSQL, etc.)
- Less boilerplate code

**Traditional approach vs. ORM approach:**

**Traditional (SQL):**
```python
cursor.execute("INSERT INTO products (name, price) VALUES (?, ?)", ("Laptop", 999.99))
```

**ORM approach:**
```python
product = Product(name="Laptop", price=999.99)
session.add(product)
session.commit()
```

## 2. SQLModel Basics

**SQLModel** is a library created by the same author as FastAPI. It combines:
- **SQLAlchemy**: A powerful ORM for Python
- **Pydantic**: A data validation and settings management library

Let's install SQLModel first:

In [ ]:
!pip install sqlmodel

Let's start with a simple example of a SQLModel model:

In [ ]:
from typing import Optional
from sqlmodel import Field, SQLModel

# Define a simple model
class SimpleProduct(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    price: float

# Create an instance of the model
product = SimpleProduct(name="Laptop", price=999.99)
print(product)
print(f"Product Name: {product.name}")
print(f"Product Price: ${product.price:.2f}")
print(f"Product ID: {product.id}")

### Key Components of a SQLModel model:

1. **Class inheritance**: `class SimpleProduct(SQLModel, table=True):`
   - Inherits from `SQLModel`
   - `table=True` parameter tells SQLModel this class represents a database table

2. **Type annotations**: `name: str`, `price: float`
   - Python type hints define the column types
   - Used for validation and schema generation

3. **Field options**: `id: Optional[int] = Field(default=None, primary_key=True)`
   - `Optional[int]` means the field can be `None`
   - `primary_key=True` designates this field as the primary key
   - `default=None` sets the default value for new instances

## 3. Creating Our Product Model

Now let's implement the actual `Product` model used in our inventory management system:

In [ ]:
from typing import Optional
from sqlmodel import Field, SQLModel
from datetime import datetime

class Product(SQLModel, table=True):
    """Product model representing items in inventory"""
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(index=True)
    description: str
    price: float
    stock_quantity: int
    category: str = Field(index=True)
    created_at: datetime = Field(default_factory=datetime.now)
    updated_at: datetime = Field(default_factory=datetime.now)

    def __repr__(self):
        return f"<Product(id={self.id}, name='{self.name}', price=${self.price:.2f}, stock={self.stock_quantity})>"

# Create a sample product
laptop = Product(
    name="MacBook Pro",
    description="Powerful laptop for developers",
    price=1999.99,
    stock_quantity=5,
    category="Electronics"
)

print(laptop)

## 4. Understanding Model Fields and Options

Let's examine each field in our `Product` model:

### 4.1. Primary Key Field

```python
id: Optional[int] = Field(default=None, primary_key=True)
```

- `Optional[int]`: The field type is integer, but can be `None` (before saving to database)
- `primary_key=True`: This field is the primary key of the table
- `default=None`: New instances have `id=None` until saved to the database

### 4.2. Indexed Fields

```python
name: str = Field(index=True)
category: str = Field(index=True)
```

- `index=True`: Creates a database index on this field
- Indexes make searching by these fields much faster
- We index `name` and `category` because we'll frequently search by them

### 4.3. Date/Time Fields

```python
created_at: datetime = Field(default_factory=datetime.now)
updated_at: datetime = Field(default_factory=datetime.now)
```

- `default_factory=datetime.now`: Calls the function to get the current time when creating a new instance
- `created_at`: Stores when the product was first added
- `updated_at`: Should be updated whenever the product is modified

### 4.4. String Representation

```python
def __repr__(self):
    return f"<Product(id={self.id}, name='{self.name}', price=${self.price:.2f}, stock={self.stock_quantity})>"
```

- `__repr__`: Defines how the object is represented as a string
- Used for debugging and when printing the object
- Shows the most important attributes of the product

## 5. Working with Model Instances

Let's explore how to work with our `Product` instances:

In [ ]:
# Create a few products
products = [
    Product(
        name="Laptop",
        description="A powerful laptop for work",
        price=1299.99,
        stock_quantity=10,
        category="Electronics"
    ),
    Product(
        name="Smartphone",
        description="Latest model with high-end camera",
        price=799.99,
        stock_quantity=15,
        category="Electronics"
    ),
    Product(
        name="Coffee Mug",
        description="Ceramic mug for hot beverages",
        price=12.99,
        stock_quantity=50,
        category="Kitchenware"
    )
]

# Access attributes
for i, product in enumerate(products, 1):
    print(f"Product {i}:")
    print(f"  Name: {product.name}")
    print(f"  Description: {product.description}")
    print(f"  Price: ${product.price:.2f}")
    print(f"  Stock: {product.stock_quantity}")
    print(f"  Category: {product.category}")
    print(f"  Created at: {product.created_at}")
    print()

### 5.1. Modifying a Product

In [ ]:
# Get the first product (Laptop)
laptop = products[0]

# Show the current price
print(f"Current price of {laptop.name}: ${laptop.price:.2f}")
print(f"Current stock of {laptop.name}: {laptop.stock_quantity}")

# Modify the product
laptop.price = 1199.99  # Apply a discount
laptop.stock_quantity -= 2  # Decrease stock by 2 units
laptop.updated_at = datetime.now()  # Update the timestamp

# Show the updated product
print("\nAfter modification:")
print(f"New price of {laptop.name}: ${laptop.price:.2f}")
print(f"New stock of {laptop.name}: {laptop.stock_quantity}")
print(f"Updated at: {laptop.updated_at}")

### 5.2. Using Model Validation

One of the benefits of SQLModel is that it validates data types:

In [ ]:
# Try creating a product with invalid data
try:
    invalid_product = Product(
        name="Invalid Product",
        description="This product has an invalid price",
        price="not a number",  # This should be a float, not a string
        stock_quantity=5,
        category="Test"
    )
except Exception as e:
    print(f"Error creating product: {e}")

### 5.3. Converting to Dictionary

SQLModel models can be easily converted to dictionaries, which is useful for serialization:

In [ ]:
# Convert a product to a dictionary
coffee_mug = products[2]
coffee_mug_dict = coffee_mug.dict()

# Display the dictionary
import json
print(json.dumps(coffee_mug_dict, indent=2, default=str))

## Summary

In this notebook, we've learned:

1. **What is an ORM** and why it's useful for database interactions
2. **SQLModel basics** - combining SQLAlchemy and Pydantic
3. **Creating data models** with fields, types, and constraints
4. **Working with model instances** - creating, modifying, and validating

This knowledge forms the foundation of our inventory management system. In the next notebooks, we'll explore how to connect to a database, perform CRUD operations, and build a command-line interface.